### Logistic Regression using PyTorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Generate synthetic binary data
X = 2 * np.random.rand(100, 1)  # generates numbers uniformly distributed between 0 and 1
y = (4 + 3 * X + np.random.randn(100, 1) > 0.5).astype(int)  # generates binary target data

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define the logistic regression model
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim=1, output_dim=1):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)  # creates a linear transformation: y = xW^T + b

    def forward(self, x):
        return torch.sigmoid(self.linear(x))  # applies the sigmoid function to output probabilities

# Initialize the model, loss function, and optimizer
model = LogisticRegressionModel()
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.SGD(model.parameters(), lr=0.01)  # model.parameters(): returns an iterator over all the parameters (weights and biases)

# Training the model
num_epochs = 1000
model.train()  # Set the model to training mode
for epoch in range(num_epochs):
    # Forward pass
    y_pred_train = model(X_train_tensor)
    loss = criterion(y_pred_train, y_train_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()   # gradients are reset to zero before the backward pass, so that the gradients are correctly computed for the current batch of data
    loss.backward()     # computes the gradient of the loss with respect to the model parameters
    optimizer.step()    # updates the model parameters using the computed gradients

# Make predictions on the test set
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # No need to track gradients for predictions
    y_pred_test = model(X_test_tensor).detach().numpy()
    y_pred_test = (y_pred_test > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate performance metrics on the test set
accuracy = accuracy_score(y_test, y_pred_test)
print(f'Accuracy on test set: {accuracy:.4f}')